In [1]:
import pandas as pd
from datetime import datetime
from sklearn import preprocessing
import numpy as np
import pickle

data = pd.read_csv("ml_data.csv")

# Data Pre-Processing

In [2]:
# Splitting the timestamp into their respective date and time values
times = data["timestamp"].tolist()
date = [time.split("T")[0] for time in times]
time = [time.split("T")[1] for time in times]
data["date"] = date
data["time"] = time

In [3]:
# Reordering the columns and removing the "timestamp" column
column = data.columns.tolist()
column = [
    'timestamp',
    'date',
    'time',
    'temp',
    'feels_like',
    'dew',
    'humidity',
    'precip',
    'precip_prob',
    'snow_depth',
    'windspeed',
    'cloud_cover',
    'visibility',
    'conditions',
    'total_rides'
]
data = data[column]
data.drop(columns=["timestamp"], inplace = True);
data

date      time  temp  feels_like   dew  humidity  precip  \
0     2022-01-01  00:00:00  -2.0        -5.6 -12.8     43.53     0.0   
1     2022-01-01  01:00:00  -2.0        -7.0 -13.4     41.30     0.0   
2     2022-01-01  02:00:00  -2.2        -7.2 -12.3     45.66     0.0   
3     2022-01-01  03:00:00  -2.6        -6.9 -13.1     44.49     0.0   
4     2022-01-01  04:00:00  -2.6        -6.5 -14.0     41.32     0.0   
...          ...       ...   ...         ...   ...       ...     ...   
4339  2022-06-30  19:00:00  24.0        24.0  18.2     70.18     0.0   
4340  2022-06-30  20:00:00  23.9        23.9  18.4     71.34     0.0   
4341  2022-06-30  21:00:00  23.6        23.6  18.5     72.95     0.0   
4342  2022-06-30  22:00:00  23.5        23.5  19.4     77.63     0.0   
4343  2022-06-30  23:00:00  23.5        23.5  20.0     80.78     0.0   

      precip_prob  snow_depth  windspeed  cloud_cover  visibility  \
0               0           0        9.5          0.0        16.0   
1               0           0       15.6         15.6        16.0   
2               0           0       15.7          0.0        16.0   
3               0           0       11.6          0.0        16.0   
4               0           0       10.4          7.2        16.0   
...           ...         ...        ...          ...         ...   
4339            0           0       15.1         62.3        16.0   
4340            0           0       13.7         88.8        16.0   
4341            0           0       16.6         88.8        16.0   
4342            0           0       11.5         92.5        16.0   
4343            0           0       16.1         91.8        16.0   

            conditions  total_rides  
0                Clear        28312  
1                Clear        31707  
2                Clear        28068  
3                Clear        24288  
4                Clear        17081  
...                ...          ...  
4339  Partially cloudy        24513  
4340  Partially cloudy        21560  
4341  Partially cloudy        24339  
4342          Overcast        22820  
4343          Overcast        18602  

[4344 rows x 14 columns]

In [4]:
# Getting the unique values in the "date", "time", and "conditions" columns and sorting them
dates = list(set(data["date"].tolist()))
times = list(set(data["time"].tolist()))
conditions = list(set(data["conditions"].tolist()))

dates.sort()
times.sort()
conditions.sort()

# Need to one-hot encode "conditions" and integer-encode dates and times
dates_mapping = {}
times_mapping = {}

for i, date in enumerate(dates):
    dates_mapping[date] = i
    
for i, time in enumerate(times):
    times_mapping[time] = i
    
# Integer encoding the "date" and "time" columns
data["date"] = data["date"].replace(dates_mapping)
data["time"] = data["time"].replace(times_mapping)

# Setting the categorical feature columns to be of type "object"
categorical_feature = ["conditions"]
data[categorical_feature] = data[categorical_feature].astype("object")

# One-hot encoding the "conditions" column
data_one_hot = pd.get_dummies(data)

# Displaying the first 5 rows of the one-hot encoded dataset
data_one_hot.head()

date  time  temp  feels_like   dew  humidity  precip  precip_prob  \
0     0     0  -2.0        -5.6 -12.8     43.53     0.0            0   
1     0     1  -2.0        -7.0 -13.4     41.30     0.0            0   
2     0     2  -2.2        -7.2 -12.3     45.66     0.0            0   
3     0     3  -2.6        -6.9 -13.1     44.49     0.0            0   
4     0     4  -2.6        -6.5 -14.0     41.32     0.0            0   

   snow_depth  windspeed  ...  conditions_Clear  conditions_Overcast  \
0           0        9.5  ...                 1                    0   
1           0       15.6  ...                 1                    0   
2           0       15.7  ...                 1                    0   
3           0       11.6  ...                 1                    0   
4           0       10.4  ...                 1                    0   

   conditions_Partially cloudy  conditions_Rain|Overcast  \
0                            0                         0   
1                            0                         0   
2                            0                         0   
3                            0                         0   
4                            0                         0   

   conditions_Rain|Partially cloudy  conditions_Snow  \
0                                 0                0   
1                                 0                0   
2                                 0                0   
3                                 0                0   
4                                 0                0   

   conditions_Snow|Overcast  conditions_Snow|Partially cloudy  \
0                         0                                 0   
1                         0                                 0   
2                         0                                 0   
3                         0                                 0   
4                         0                                 0   

   conditions_Snow|Rain|Overcast  conditions_Snow|Rain|Partially cloudy  
0                              0                                      0  
1                              0                                      0  
2                              0                                      0  
3                              0                                      0  
4                              0                                      0  

[5 rows x 23 columns]

In [5]:
data_one_hot.columns

Index(['date', 'time', 'temp', 'feels_like', 'dew', 'humidity', 'precip',
       'precip_prob', 'snow_depth', 'windspeed', 'cloud_cover', 'visibility',
       'total_rides', 'conditions_Clear', 'conditions_Overcast',
       'conditions_Partially cloudy', 'conditions_Rain|Overcast',
       'conditions_Rain|Partially cloudy', 'conditions_Snow',
       'conditions_Snow|Overcast', 'conditions_Snow|Partially cloudy',
       'conditions_Snow|Rain|Overcast',
       'conditions_Snow|Rain|Partially cloudy'],
      dtype='object')

In [6]:
# Scaling the descriptive features (normalization)
# from sklearn import preprocessing
# descriptive_features = ['temp','feels_like','dew','humidity','precip','snow_depth','windspeed','cloud_cover','visibility']

# for feature in descriptive_features:
#     values = data_one_hot[feature].values
#     values = np.round_(preprocessing.MinMaxScaler().fit_transform(values.reshape(-1, 1)).ravel(),1)
#     data_one_hot[feature] = values
    
# data_one_hot.head()

# Predictive Modelling

In [7]:
# Splitting the data and target values
data_df = data_one_hot.drop(columns = "total_rides")
data = data_df.values
target = data_one_hot["total_rides"].values

# Generating training and testing sets
from sklearn.model_selection import train_test_split

d_train, d_test, t_train, t_test = train_test_split(data,
                                                   target,
                                                   test_size = 0.2,
                                                   shuffle = True,
                                                   random_state = 999)

### Decision Tree 

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, GridSearchCV

dt_regressor = DecisionTreeRegressor() 

# Initialising a stratified 5-fold cross-validation method
cv_method = KFold(n_splits = 5)

# Defining the parameters for feature selection and the DT classifer
DT_params = {'criterion': ['friedman_mse', 'poisson'],
             'max_depth': [3, 4, 5, 10, 15], 
             'min_samples_split': [2, 5, 10],
             'splitter': ['best', 'random'],
             'max_features': ['auto', 'sqrt', 'log2', None]}

# Defining the grid search for the DT
DT_gridsearch = GridSearchCV(estimator = dt_regressor,
                             param_grid = DT_params,
                             cv = cv_method,
                             refit = True,
                             n_jobs = -2,
                             scoring = 'r2', 
                             verbose = 1)

# Fitting the DT grid search with the training data and target labels
DT_gridsearch.fit(d_train, t_train);

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


In [9]:
DT_gridsearch.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_split': 5,
 'splitter': 'best'}

In [10]:
DT_gridsearch.best_score_

0.665037826353317

### Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression

lr_regressor = LinearRegression() 

lr_regressor.fit(d_train, t_train);
lr_regressor.score(d_test, t_test)

0.45606927761432037

### K-Neighbors Regressor

In [12]:
from sklearn.neighbors import KNeighborsRegressor

knn_regressor = KNeighborsRegressor() 

# Initialising a stratified 5-fold cross-validation method
cv_method = KFold(n_splits = 5)

# Defining the parameters for feature selection and the KNN regressor
knn_params = {'n_neighbors': [5, 10, 20, 50], 
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'leaf_size': [5, 15, 30, 50, 100],
              'p': [1, 2]}

# Defining the grid search for the KNN
knn_gridsearch = GridSearchCV(estimator = knn_regressor,
                             param_grid = knn_params,
                             cv = cv_method,
                             refit = True,
                             n_jobs = -2,
                             scoring = 'r2', 
                             verbose = 1)

# Fitting the KNN grid search with the training data and target labels
knn_gridsearch.fit(d_train, t_train);

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


In [13]:
knn_gridsearch.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 100,
 'n_neighbors': 5,
 'p': 1,
 'weights': 'distance'}

In [14]:
knn_gridsearch.best_score_

0.5429406222515792

### Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor

# lin_regressor = LinearRegression()
rf_regressor = RandomForestRegressor()

# Initialising a stratified 5-fold cross-validation method
cv_method = KFold(n_splits = 5)

# Defining the parameters for feature selection and the Logistic Regressor
rf_params = {'n_estimators': [5, 10, 30, 50, 100], 
            'max_depth': [4, 5, 6],
            'min_samples_split': [2, 5, 10]}

# Defining the grid search for the KNN
rf_gridsearch = GridSearchCV(estimator = rf_regressor,
                             param_grid = rf_params,
                             cv = cv_method,
                             refit = True,
                             n_jobs = -2,
                             scoring = 'r2', 
                             verbose = 1)

# Fitting the KNN grid search with the training data and target labels
rf_gridsearch.fit(d_train, t_train);

Fitting 5 folds for each of 45 candidates, totalling 225 fits


In [16]:
rf_gridsearch.best_params_

{'max_depth': 6, 'min_samples_split': 2, 'n_estimators': 100}

In [17]:
rf_gridsearch.best_score_ 

0.6898051060736419

# Model Evaluation

In [18]:
from sklearn.model_selection import cross_val_score

cv_method_ttest = KFold(n_splits = 10)

DT_cv_results = cross_val_score(estimator = DT_gridsearch.best_estimator_,
                                X = d_test,
                                y = t_test,
                                cv = cv_method_ttest,
                                n_jobs = -2,
                                scoring = 'r2')

DT_cv_results.mean()

0.6026138888941998

In [19]:
knn_cv_results = cross_val_score(estimator = knn_gridsearch.best_estimator_,
                                X = d_test,
                                y = t_test,
                                cv = cv_method_ttest,
                                n_jobs = -2,
                                scoring = 'r2')

knn_cv_results.mean()

0.33188590371409815

In [20]:
lr_cv_results = cross_val_score(estimator = lr_regressor,
                                X = d_test,
                                y = t_test,
                                cv = cv_method_ttest,
                                n_jobs = -2,
                                scoring = 'r2')

lr_cv_results.mean()

0.42179503760823833

In [21]:
rf_cv_results = cross_val_score(estimator = rf_gridsearch.best_estimator_,
                                X = d_test,
                                y = t_test,
                                cv = cv_method_ttest,
                                n_jobs = -2,
                                scoring = 'r2')

rf_cv_results.mean()

0.6518671941698536

Show the feature engineering part in the report.

Compare base line model to advanced models i.e. DT vs RF.

Use R2 value instead of MAE to justify the performance of the model.